## Welcome to Week 4, Single Cell RNA (cont.)!

### This week, we're going to go a bit deeper into scRNA analysis, such as how to interact with Seurat objects, add additional datatypes including CITE-seq and TCR/BCR-seq data, and create custom, publication-ready plots.

We'll continue to use Scanpy, which has some nice capabilities for multi-modal data analysis. The two datatypes we will be working with today at **CITE-seq** and **TCR/BCR-seq** data. The main idea of both is that additional information about the cell is captured using the same cell barcode from reverse transcription so that multiple types of data can be assigned to the same cell. CITE-seq is a method for capturing surface protein information using oligo-conjugated antibodies developed at the New York Genome Center. Here antibodies are conjugated to oligos which contain two important sequences: an antibody specific barcode which is used to quantify surface protein levels in individual cells and a capture sequence (either poly-A sequence or a 10X specific capture sequence) which enables the antibody oligo to be tagged with the cell barcode during reverse transcription. You can look at more details in the publication here:  
* https://www.ncbi.nlm.nih.gov/pubmed/28759029  

Oligo-conjugated anitbodies compatible with 10X scRNA (both 5' and 3') are commercially available from BioLegend (https://www.biolegend.com/en-us/totalseq) and can also be used to multiplex different samples in the same 10X capture. This works by using an antibody which recognizes a common surface antigen and using the antibody barcode to distinguish between samples, a process known as **cell hashing**:
* https://www.ncbi.nlm.nih.gov/pubmed/30567574

We won't be using hashtag data today, but many of the same strategies apply and feel free to reach out if you are interested in learning more!  

The second data type we will be working with is TCR/BCR sequencing data. T and B cells express a highly diverse repertoire of transcripts resulting from V(D)J recombination - the T cell receptor (TCR) in T cells and immunoglobulin (Ig) or BCR in B cells. Daughter cells will share the same TCR/BCR sequence, allowing this sequence to be used to track clonal cell populations over time and space, as well as infer lineage relationships. TCR/BCR sequences are amplified from the cDNA library in the 5' immune profiling 10X kit, allowing these sequences to be matched to the gene expression library from the same cell. For more details, see the 10X website:
* https://www.10xgenomics.com/products/vdj/  

For both of these applications, we'll be following this tutorial:
* https://scanpy-tutorials.readthedocs.io/en/multiomics/cite-seq/pbmc5k.html

### Import Statements

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from collections import Counter, defaultdict
from scipy import stats as scistats

import scrublet as scr
import scipy.io

%matplotlib inline

In [ ]:
# you'll need to change these for yourself

path = '/Users/kevin/changlab/covid19/3_scRNA/data/filtered_feature_bc_matrix/'
figpath = '/Users/kevin/changlab/covid19/4_scRNA-part-2/figures/'

# lets set the default figure settings

sc.settings.set_figure_params(dpi_save=300)
sc.settings.figdir = figpath

In [ ]:
# helpful plotting functions, "sax" or "simple ax" and "prettify ax" or "pax"

def pax(ax):
    mpl.rcParams['font.sans-serif'] = 'Helvetica'
    for spine in ax.spines.values():
        spine.set_color('k')
    ax.set_frameon=True
    ax.patch.set_facecolor('w')
    ax.tick_params(direction='out', color = 'k', length=5, width=.75, pad=8)
    ax.set_axisbelow(True)
    ax.grid(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    mpl.rcParams['font.sans-serif'] = 'Helvetica'

def sax(figsize=(6,6)):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    pax(ax)
    return fig, ax

def sf(fig, fn, formats=['png'], dpi=300, figpath=figpath):
    for f in formats:
        fig.savefig(figpath + fn + '.' + f, dpi=dpi, bbox_inches='tight')

### First, go back to the week three notebook, re-run everything, and save the output so you can just re-import the procssed dataset here.

Or, you can use the file that I outputted to have the same input.  I've included the code that I ran to generate it below.

In [ ]:
# # process with scrublet
# print('processing with scrublet')
# counts_matrix = scipy.io.mmread(path + '/matrix.mtx.gz').T.tocsc()
# cells = pd.read_csv(path + '/barcodes.tsv.gz', sep='\t', header=None, names=['barcode'])
# cells = cells.set_index('barcode', drop=False)

# scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.08)
# doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
#                                                           min_cells=3, 
#                                                           min_gene_variability_pctl=85, 
#                                                           n_prin_comps=30)
# predicted_doublets = scrub.call_doublets(threshold=0.25)
# cells['doublet_score'] = doublet_scores
# cells['predicted_doublet'] = predicted_doublets

# # import data
# print('importing data')
# gex = sc.read_10x_mtx(path, gex_only=True)
# gex.obs['doublet_score'] = cells.loc[gex.obs.index, 'doublet_score']
# gex.obs['predicted_doublet'] = cells.loc[gex.obs.index, 'predicted_doublet']

# # preliminary processing
# print('preliminary processing')
# sc.pp.filter_cells(gex, min_genes=200)
# sc.pp.filter_genes(gex, min_cells=3)
# mito_genes = gex.var_names.str.startswith('MT-')
# gex.obs['percent_mito'] = np.sum(
#     gex[:, mito_genes].X, axis=1).A1 / np.sum(gex.X, axis=1).A1
# gex.obs['n_counts'] = gex.X.sum(axis=1).A1
# gex = gex[gex.obs.n_genes >= 500, :]
# gex = gex[gex.obs.percent_mito < 0.1, :]
# sc.pp.normalize_total(gex, target_sum=1e4)
# sc.pp.log1p(gex)
# gex.raw = gex

# # dimensionality reduction
# print('secondary processing')
# sc.pp.highly_variable_genes(gex, n_top_genes=2000)
# gex = gex[:, gex.var.highly_variable]
# sc.pp.regress_out(gex, ['n_genes'])
# sc.pp.scale(gex, max_value=10)
# sc.tl.pca(gex, svd_solver='arpack', n_comps=50)
# sc.pp.neighbors(gex, n_neighbors=10, n_pcs=50, random_state=1)
# sc.tl.leiden(gex, random_state=1, resolution=.4)
# sc.tl.umap(gex)
# new_cluster_names = ['Mono_CD14', #0
#                     'CD4 T', #1
#                     'B', #2
#                     'CD8 T', #3
#                     'NK', #4
#                     'CD8 Tem', #5
#                     'Mono_FCGR3A', #6
#                     'Und1_Doublets', #7
#                     'cDC', #8
#                     'gd T', #9 gamma delta t cells
#                     'pDCs', #10
#                     'Platelets', #11
#                     'Plasma B', #12
#                     'Und2', #13
#                     ]
# gex.rename_categories('leiden', new_cluster_names)

# # plot things
# print('plotting')
# # plot1
# fig = sc.pl.umap(gex, color=['leiden'],
#                 legend_fontsize = 8,
#                 legend_loc = 'on data', return_fig=True)
# fig.savefig(figpath + '0_leiden-clustering-renamed.png', dpi=300, bbox_inches='tight')
# # plot2
# genes_to_plot = ['predicted_doublet','n_genes',
#                 'n_counts','percent_mito']
# fig = sc.pl.umap(gex, color=genes_to_plot, use_raw=True,
#                 sort_order=True, ncols=2, return_fig=True)
# fig.savefig(figpath + '0_umap-metadata.png', dpi=300, bbox_inches='tight')
# # plot3
# genes_to_plot = ['CD3G','CD4','CD8A',
#                  'TRDV2','KLRB1','NKG7',
#                  'CD14','FCGR3A','FCER1A',
#                  'MS4A1','JCHAIN','PPBP',
#                 ]

# fig = sc.pl.umap(gex, color=genes_to_plot, use_raw=True,
#                 sort_order=True, ncols=3,return_fig=True, color_map='Reds')
# fig.savefig(figpath + '0_umap-gene-expression.png', dpi=300, bbox_inches='tight')

# # save the results
# gex.write(figpath + 'scrna_wk3_processed.h5ad', compression='gzip')

In [ ]:
# import the data

# reminder that you'll need to change the path to this

gex = sc.read_h5ad(figpath + 'scrna_wk3_processed.h5ad')
gex

In [ ]:
# make sure that everything looks good
genes_to_plot = ['CD3G','CD4','CD8A',
                 'TRDV2','KLRB1','NKG7',
                 'CD14','FCGR3A','FCER1A',
                 'MS4A1','JCHAIN','PPBP',
                ]

fig = sc.pl.umap(gex, color=genes_to_plot, use_raw=True,
                sort_order=True, ncols=3,return_fig=True, color_map='Reds')
plt.show()

fig = sc.pl.umap(gex, color=['leiden'],
                legend_fontsize = 8,
                legend_loc = 'on data', return_fig=True)
plt.show()

### CITE-seq Analysis

In [ ]:
# first, read in the cite seq information
# remember that gex_only=False will let you read them both in

data = sc.read_10x_mtx(path, gex_only=False)
data

In [ ]:
# what cite seq features do we have?
# how many genes?
# how many cite-seq?


In [ ]:
# rename the antibody capture genes
# get rid of the "_TotalSeqC" part of the name just to make our lives easier
# e.g. CD3_TotalSeqC to CD3


In [ ]:
# filter this to just include cells that we analyzed previously, so the datasets will align
# you can do this with

data = data[data.obs.index.isin(gex.obs.index), :]

In [ ]:
# now lets get just the protein information, and make that its own anndata object

protein = data[:, data.var['feature_types'] == 'Antibody Capture'].copy()
protein

### Now let's break out of scanpy for a minute to inspect, normalize, and scale this data on our own

Scanpy seems to be developing some functions specifically for protein data, but hasn't yet implemented them.  But this isn't a problem! We can do things on our own, and transform the data into a format that scanpy wants.

**We're going to break this down in a few steps:**

1. get the raw antibody count data from the protein anndata object.  
2. compute the centered log ratio (CLR) of antibody counts (this is different than for RNA!) - more notes on this below.  
3. scale the data to be mean centered and have unit variance (i.e., z-normalization).  This is the same as for RNA.   
4. save the CLR normalized antibody counts as the raw data of the protein object, and the scaled data as the (normal) data of the protein object, which will be used for dimensionality reduction.  

Now, in terms of what the actual normalizations are: we're going to do this with the .apply() function with dataframes.  I'm providing an example for how to you would do the depth normalization that you'd normally do for RNA-seq below, but you should play around on your own with implementing the normalizations in 2 and 3.

**Normalization methods:**

* depth normalization (as a comparison).  For a cell, divide the counts for each gene/antibody by the sum of all gene/antibody counts for that cell, then multiply by some scaling factor (e.g. 10,000).  Commonly, you would also log transform this, and add a pseudocount (say 1).  This is sometimes referred to as **log1p**.
* CLR.  For an antibody, divide the counts for each antibody by the geometric mean antibody counts across all cells, then take the natural log of this.  Similarly, you'll add a pseudocount of 1.
* z-normalization (scaling to zero mean and unit variance).  Basically, you're making all datapoints have similar distributions.  For a gene, return the count for a cell minus the mean count across all cells, divided by the standard deviation across all cells.
* clipping extremes.  You can use the np.clip() function to do this.  Basically, this will take any value lower than the lower bound in np.clip and make it equal to the lower bound, and do the same for the upper bound.  You might combine this with computing the mean and standard deviation, to clip values > 3 stds away from the mean; or np.percentile() to clip values that are less or greater than a given percentile in the data.

It's worth taking the time to look at why the CLR transformation is better than a simple log transformation.  Why?  Because antibodies aren't genes - when a gene is negative, the count is 0; when a gene is positive, the count is greater than 0.  But does this hold true with antibodies?  When an antibody is negative, the count isn't necessarily 0 - the antibody might have background!  The CLR transformation does a better job of dealing with this, by looking at the relative abundance of the antibody.

In [ ]:
# get the raw data

protein_orig = pd.DataFrame(protein.X.todense(), index=protein.obs.index, columns=protein.var.index).T

In [ ]:
# what does your data look like?

# I'd recommend first plotting the distribution of total antibody counts across all cells


# sf(fig, '1_preprocess_histogram_antibody-counts')

In [ ]:
# what if we just take a 'naive' approach to normalization?

protein_norm_depth = protein_orig.apply(lambda x: 10000 * x / x.sum(), axis=0)
protein_norm_depth = np.log(protein_norm_depth + 1)

# plot the distribution of counts for all of these

fig = plt.figure(figsize=(10,10))
axes = [fig.add_subplot(5,4,i+1) for i in range(len(protein_norm_depth.index))]
xlim = [protein_norm_depth.min().min(), protein_norm_depth.max().max()]

bins = np.linspace(xlim[0], xlim[1], 100)

for ix, p in enumerate(protein_orig.index):
    ax = axes[ix]
    pax(ax)
     
    vals = protein_norm_depth.loc[p]

    ax.hist(vals, bins=bins)
    ax.set_title(p, size=16)
    ax.set_xlim(xlim)

fig.tight_layout()
plt.show()

sf(fig, '1_preprocess_log1p-distributions')

In [ ]:
# now lets compare this with the CLR approach
def clr(x, pseudo=1):
    x = x + pseudo
    geo_mean = scistats.gmean(x)
    return np.log(x / geo_mean)

protein_norm_clr = protein_orig.apply(clr, axis=1)
protein_norm_clr.head()

# plot the distribution of counts for all of these



# sf(fig, '1_preprocess_clr-distributions')

In [ ]:
# now lets compare the two with a scatter plot


# sf(fig, '1_preprocess_scatter-norm-methods')

In [ ]:
# now scale this to unit variance
# see https://en.wikipedia.org/wiki/Feature_scaling under z-normalization


# also, clip extremes - clip anything less than -10 and above 10


In [ ]:
# plot the distribution of counts for all of the scaled data
# note how the distributions are relatively similar


# sf(fig, '1_preprocess_scaled_clr-distributions')

In [ ]:
# what if we want to make a scatter plot of one CD4 vs CD8a?
# compare the depth-normalized vs CLR normalized counts
# make it once with depth-normalized counts and once with CLR normalized


# sf(fig, '1_preprocess_scatter_log1p_cd4-8')

In [ ]:
# what if we want to make a scatter plot of one antibody?


# sf(fig, '1_preprocess_scatter_clr_cd4-8')

### Now go back to scanpy

Let's save the protein_norm_clr values as the raw data in protein, and the protein_scaled values in the data slot of protein.  Let's also exclude the control proteins from the main data slot.

In [ ]:
protein = data[:, data.var['feature_types'] == 'Antibody Capture'].copy()

protein.var['control'] = ['control' in i for i in protein.var.index]
protein.X = protein_norm_clr.T
protein.raw = protein

protein.X = protein_scaled.T
protein = protein[:, ~protein.var['control']]

protein

In [ ]:
protein.var

In [ ]:
protein_genes = ['CD3D','CD19','PTPRC',
                'CD4','CD8A','CD14','FCGR3A',
                'NCAM1','IL2RA','PTPRC',
                'PDCD1','TIGIT','IL7R','FUT4']
protein.var['rna_name'] = protein_genes
name_dict = dict(zip(protein.var.index, protein.var['rna_name']))
protein.var.head()

In [ ]:
sc.pp.pca(protein, n_comps=len(protein.var)-1)
sc.pp.neighbors(protein, n_neighbors=30, n_pcs=len(protein.var)-1)
sc.tl.leiden(protein, key_added="protein_leiden", resolution=.33)

In [ ]:
sc.tl.umap(protein)

In [ ]:
genes_to_plot = protein.var.index.tolist() + ['protein_leiden']

fig = sc.pl.umap(protein, color=genes_to_plot,
                sort_order=True, ncols=4,return_fig=True, color_map='Blues', use_raw=True,
                vmin='p5', vmax='p99.9')
fig.set_size_inches(12,12)
sf(fig,'2_umap_with_cite-clustering')
plt.show()

### Now let's integrate this with the RNA data

I'm going to do this a little fast and loose because I think that scanpy hasn't yet fully implemented the CITE-seq stuff too well.  Basically, we're going to add the umap coordinates and clustering information from the RNA processed data to the protein-processed data, and vice versa.

In [ ]:
# add gex to protein
protein.obsm['RNA_umap'] = gex[protein.obs.index].obsm['X_umap']
protein.obs['rna_leiden'] = gex.obs.loc[protein.obs.index, 'leiden']

# add protein to gex
# I'll leave you do to this

In [ ]:
# now, let's plot the cite-seq information on top of the rna clusters

genes_to_plot = protein.var.index.tolist() + ['rna_leiden']

fig = sc.pl.embedding(protein, 'RNA_umap', color=genes_to_plot,
                sort_order=True, ncols=4,return_fig=True, color_map='Blues', use_raw=True,
                vmin='p5', vmax='p99.9', legend_fontsize=8)
fig.set_size_inches(12,12)
sf(fig,'3_RNA-umap_with_CITE-counts')
plt.show()

In [ ]:
# and, let's plot some rna-seq information on top of the cite clusters
# I'll leave you to do this one

# sf(fig,'3_CITE-umap_with_RNA-counts')

### Now, let's plot RNA information against CITE information to see how they compare.

In [ ]:
# first, get the metadata from the scanpy .obs dataframe

meta = gex.obs
meta.head()

In [ ]:
# and add in the umap coordinates from the RNA
meta['umap_1'] = gex.obsm['X_umap'][:, 0]
meta['umap_2'] = gex.obsm['X_umap'][:, 1]

# now add in the umap coordinates from the CITE-seq
meta['umap-cite_1'] = protein[meta.index].obsm['X_umap'][:, 0]
meta['umap-cite_2'] = protein[meta.index].obsm['X_umap'][:, 1]

meta.head()

In [ ]:
# here's two helper functions to get gene/protein expression information

def get_gene_expression(gene, adata=gex, undo_log=False, cells=''):
    gene_ix = adata.raw.var.index.get_loc(gene)
    vals = adata.raw.X[:, gene_ix].toarray().ravel()
    
    if undo_log:
        vals = np.exp(vals) - 1
    
    vals = pd.Series(vals, index=adata.obs.index)
        
    return vals

def get_protein_expression(gene, data=protein_norm_clr):
    vals = protein_norm_clr.loc[gene]
    
    return vals

In [ ]:
# make a scatter plot of RNA expression vs CITE-seq counts

for gene in protein.var.index:

    rna_vals = get_gene_expression(name_dict[gene])
    protein_vals = get_protein_expression(gene)

    sf(fig, '4_scatter_rna-cite_' + gene)

In [ ]:
# plot the RNA and CITE counts on top of the UMAP from the RNA data

for gene in protein.var.index:

    rna_vals = get_gene_expression(name_dict[gene])
    protein_vals = get_protein_expression(gene)

    fig = plt.figure(figsize=(10,5))

    # plot RNA
    # plot PROTEIN

    sf(fig, '4_umap_rna-cite_' + gene)